In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("combined.csv")

def convertToFloat(x):
    x=x.strip()
    if x=="-" or x=="NR" or x=='' or x=="." or x=="" or x==' ':
        return 0.01
    else:
        
        return float(x)

def convertToFloat2(x):
    x=x.strip()
    if x=="-" or x=="NR" or x=='' or x=="." or x=="" or x==' ':
        return 0
    else:
        
        return float(x)


df["rain"]=df["Rain"].apply(convertToFloat2)
import datetime
df["month"]=df["Date"].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").month)
df["year"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").year)
df["day"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").day)




def SparseMeasure(x):
    if x ==1 or x==3:
        return 0
    elif x==2 or x==4 or x==12:
        return 1
    elif x==5 or x==6:
        return 2
    elif x==7 or x==10:
        return 3
    else: 
        return 4

def meandictfunc(row):
    daymonth=str(row["day"])+"_"+str(row["month"])
    meanval=meandict[daymonth]
    return meanval

def isRain2(x):
    if x>100:
        return 5
    if x>50:
        return 4
    if x>25:
        return 3
    if x>2:
        return 2
    elif x!=0:
        return 1
 
    else:
        return 0
def get_season(x):
    if x>=1 or x<=4:
        return 0
    elif x>=5 or x<=9:
        return 2
    elif x>9 or x<12:
        return 1
    else:
        return 0
def isSparse(row):
    daymonth=str(row["day"])+"_"+str(row["month"])
    val=meandict[daymonth]
    if val<1:
        return 0
    elif val>=1 or val<2.5:
        return 4
    elif val>=2.5 or val<4:
        return 8
    else:
        return 12

def lastseventh(row):
    daymonth=str(row["day"])+"_"+str(row["month"])
    val=MonDay[daymonth][-7]
    if val==0:
        return 0
    else:
        return np.log(val)
    return val
def lastsixth(row):
    daymonth=str(row["day"])+"_"+str(row["month"])
    val=MonDay[daymonth][-6]
    if val==0:
        return 0
    else:
        return np.log(val)
    return val
    
def lastfifth(row):
    daymonth=str(row["day"])+"_"+str(row["month"])
    val=MonDay[daymonth][-5]
    if val==0:
        return 0
    else:
        return np.log(val)
    return val

 
def lastfourth(row):
    daymonth=str(row["day"])+"_"+str(row["month"])
    val=MonDay[daymonth][-4]
    if val==0:
        return 0
    else:
        return np.log(val)
    return val

def lastthrid(row):
    daymonth=str(row["day"])+"_"+str(row["month"])
    val=MonDay[daymonth][-3]
    if val==0:
        return 0
    else:
        return np.log(val)
    return val

def lastsecond(row):
    daymonth=str(row["day"])+"_"+str(row["month"])
    val=MonDay[daymonth][-2]
    if val==0:
        return 0
    else:
        return np.log(val)
    return val

def lastfirst(row):
    daymonth=str(row["day"])+"_"+str(row["month"])
    val=MonDay[daymonth][-1]
    if val==0:
        return 0
    else:
        return np.log(val)
    return val
def last(row,k):
    daymonth=str(row["District"])+"_"+str(row["day"])+"_"+str(row["month"])
    if k<len(MonDay[daymonth]):
        val=MonDay[daymonth][-k]
    else:
        val=0
    final=isRain2(val)
    return final
    


In [2]:
df.to_csv("converted.csv",index=False)

In [3]:
yearlist=df.year.unique()
MonDay={}
for year in yearlist:
    yeardf=df[df.year==year]
    for frame,row in yeardf.iterrows():
        monthday=str(row["District"])+"_"+str(row["day"])+"_"+str(row["month"])
        if monthday not in MonDay.keys():
            MonDay[monthday]=[]
        
        MonDay[monthday].append(row["rain"])

meandict={}
for key in MonDay.keys():
    themean=np.mean(np.array(MonDay[key]))
    if themean==0.0:
        meandict[key]=themean
    else:
        meandict[key]=themean

In [5]:
len(MonDay["Alangayam_10_6"])

10

In [9]:
df["is_rain"]=df["month"].apply(get_season)
# df["sparse1"]=df.apply(lastfirst,axis=1)
# df["sparse2"]=df.apply(lastsecond,axis=1)
# df["sparse3"]=df.apply(lastthrid,axis=1)
# df["sparse4"]=df.apply(lastfourth,axis=1)
# df["sparse5"]=df.apply(lastfifth,axis=1)
# df["sparse6"]=df.apply(lastsixth,axis=1)
# df["sparse7"]=df.apply(lastseventh,axis=1)
for i in range(1,9):
    sparse="sparse"+str(i)
    df[sparse]=df.apply(last,k=i,axis=1)
# df["sparse"]=df.apply(isSparse,axis=1)

In [186]:
min(df["sparse8"])

0

In [140]:
df["new2"]=df.apply(meandictfunc,axis=1)
# df["rain"]=df["rain"].apply(lambda x: np.log(x))
# df["year"]=df["year"].apply(lambda x: np.log(x))
# df["new2"].apply(lambda x: np.log(x))
# df["new"]=df["month"].apply(SparseMeasure)


In [10]:
dummydf=pd.get_dummies(df[["District","rain","month","day","year","sparse1","sparse2","sparse3","sparse4","sparse5","sparse6","sparse7","sparse8"]])
# dummydf=pd.get_dummies(df[["District","rain","month","day","year"]])


In [11]:
y=dummydf.pop("rain")
x=dummydf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X_train, X_test, y_train, y_test = train_test_split(x , y, test_size=0.33, random_state=42)

In [4]:
import xgboost as xg
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',    
                  n_estimators =3, seed = 123,max_depth=35,eta=0.3)
   
# Fitting the model
# xgb_r.fit(X_train, y_train)

In [37]:
from sklearn.metrics import mean_absolute_error
pred=xgb_r.predict(X_test)
predtrain=xgb_r.predict(X_train)
for i in range(len(pred)):
    if pred[i]<=1:
        pred[i]=0
for i in range(len(predtrain)):
    if predtrain[i]<=1:
        predtrain[i]=0
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_train,predtrain))
print(mean_squared_error(y_test,pred) )
print(mean_absolute_error(y_test,pred))


16.80917898736388
32.780520991666066
1.5726110230848696
C:\Users\om purohit\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [39]:
d={"orig":y_test,"pred_test":pred}
preddf=pd.DataFrame(d)


preddf[10:40].values

array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 1.2       ,  0.        ],
       [ 0.        ,  0.        ],
       [ 3.6       , 11.59256268],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [13.        ,  3.37072515],
       [ 3.2       ,  2.44795179],
       [45.4       ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [49.        , 22.95454025],
       [ 3.        ,  4.80077457],
       [14.8       ,  3.11541271],
       [ 2.9       ,  0.        ],
       [ 1.2       ,  0.        ],
       [ 0.        ,  0.        ],
       [40.2       , 19.51211929],
       [ 0.        ,  0.        ],
       [ 6.9       ,  3.74354482],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,

In [175]:
mean_absolute_error(preddf["orig"],preddf["pred_test"])

2.7467226615210842

In [40]:

xgb_r.save_model("modelsxgb.json")

In [6]:
loadtest = xg.XGBRegressor(objective ='reg:squarederror',    
                  n_estimators =3, seed = 123,max_depth=35,eta=0.3)
   

In [7]:
loadtest.load_model("modelsxgb.json")

In [19]:
import numpy as np
tt=[0]*29
print(tt)
loadtest.predict(np.array([np.array(tt)]))[0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


0.1742031

In [18]:
x=[1,2,3]
x.extend([4,5,6])
x

[1, 2, 3, 4, 5, 6]

In [206]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
predreg=reg.predict(X_test)
mean_absolute_error(y_test,predreg)
mean_squared_error(y_test,predreg)

70.63919713812233

In [182]:
from sklearn.svm import LinearSVR
eps = 5
svr = LinearSVR(epsilon=eps, C=0.1,)
svr.fit(X_train,y_train)
predsvr=svr.predict(X_test)
for i in range(len(predsvr)):
    if predsvr[i]<1:
        predsvr[i]=0
mean_absolute_error(y_test,predsvr)

C:\Users\om purohit\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


5.279284447484633

In [62]:
meandict["2_1"]


0.0

In [41]:
import pickle

with open('DayMon.json', 'wb') as fp:
    pickle.dump(MonDay, fp)

In [2]:
import pickle
with open("DayMon.json","rb") as fp:
    toprint=pickle.load(fp)

In [3]:
toprint.keys()

ill_15_3', 'Ponnai_15_3', 'Katpadi_16_3', 'Natrampalli_16_3', 'TCS Mill_16_3', 'ACS Mill_16_3', 'VCS Mill_16_3', 'Ponnai_16_3', 'Katpadi_17_3', 'Natrampalli_17_3', 'TCS Mill_17_3', 'ACS Mill_17_3', 'VCS Mill_17_3', 'Ponnai_17_3', 'Katpadi_18_3', 'Natrampalli_18_3', 'TCS Mill_18_3', 'ACS Mill_18_3', 'VCS Mill_18_3', 'Ponnai_18_3', 'Katpadi_19_3', 'Natrampalli_19_3', 'TCS Mill_19_3', 'ACS Mill_19_3', 'VCS Mill_19_3', 'Ponnai_19_3', 'Katpadi_20_3', 'Natrampalli_20_3', 'TCS Mill_20_3', 'ACS Mill_20_3', 'VCS Mill_20_3', 'Ponnai_20_3', 'Katpadi_21_3', 'Natrampalli_21_3', 'TCS Mill_21_3', 'ACS Mill_21_3', 'VCS Mill_21_3', 'Ponnai_21_3', 'Katpadi_22_3', 'Natrampalli_22_3', 'TCS Mill_22_3', 'ACS Mill_22_3', 'VCS Mill_22_3', 'Ponnai_22_3', 'Katpadi_23_3', 'Natrampalli_23_3', 'TCS Mill_23_3', 'ACS Mill_23_3', 'VCS Mill_23_3', 'Ponnai_23_3', 'Katpadi_24_3', 'Natrampalli_24_3', 'TCS Mill_24_3', 'ACS Mill_24_3', 'VCS Mill_24_3', 'Ponnai_24_3', 'Katpadi_25_3', 'Natrampalli_25_3', 'TCS Mill_25_3', 'AC

In [47]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [45]:
X_test.columns

Index(['month', 'day', 'year', 'sparse1', 'sparse2', 'sparse3', 'sparse4',
       'sparse5', 'sparse6', 'sparse7', 'sparse8', 'District_ACS Mill',
       'District_Alangayam', 'District_Ambur', 'District_Arakkonam',
       'District_Arcot', 'District_Gudiyatham', 'District_Katpadi',
       'District_Kaveripakkam', 'District_Melalathur', 'District_Natrampalli',
       'District_Ponnai', 'District_Sholingur', 'District_TCS Mill',
       'District_Tirupattur', 'District_VCS Mill', 'District_Vaniyambadi',
       'District_Vellore', 'District_Wallajah'],
      dtype='object')

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("converted.csv")

In [3]:
dfg=df.groupby(["District","year"]).mean().reset_index()
dfg

,District,year,rain,month,day
0,ACS Mill,2018,1.632609,11.000000,15.836957
1,ACS Mill,2019,2.486575,6.526027,15.720548
2,Alangayam,2010,2.781644,6.526027,15.720548
3,Alangayam,2011,2.627123,6.526027,15.720548
4,Alangayam,2012,2.923361,6.513661,15.756831
...,...,...,...,...,...
127,Wallajah,2015,2.616986,6.526027,15.720548
128,Wallajah,2016,1.008743,6.513661,15.756831
129,Wallajah,2017,2.004384,6.526027,15.720548
130,Wallajah,2018,1.155890,6.526027,15.720548


In [38]:
df=df[(df["year"]==2014)&(df["District"]=='Arcot')]

In [40]:
df.groupby(["District","year","month"]).mean().reset_index()

,District,year,month,rain,day
0,Arcot,2014,1,0.000000,16.0
1,Arcot,2014,2,0.000000,14.5
2,Arcot,2014,3,0.000000,16.0
3,Arcot,2014,4,0.000000,15.5
4,Arcot,2014,5,2.358065,16.0
5,Arcot,2014,6,0.000000,15.5
6,Arcot,2014,7,2.048387,16.0
7,Arcot,2014,8,3.774194,16.0
8,Arcot,2014,9,4.960000,15.5
9,Arcot,2014,10,4.119355,16.0
